![BAE logo](images/bae_logo.png)

# Hands-On Exercise 12.1: The Strategy Pattern

## Objective

In this exercise, you will use the Strategy pattern. The Strategy pattern allows the developer to encapsulate algorithms in an object under a standard interface. Once implemented, it will allow the details of the algorithm to be replaced without change to the objects that use it.

This exercise continues using the Blackjack simulation. It includes all changes made in the bonus from the previous exercise. You have been asked to make modifications to code which is not yours---a typical chore for a programmer. The automated unit tests should help you feel comfortable making changes and hence, you should experience what the benefits of TDD feel like.

Specifically, the task is to allow multiple different strategies for shuffling a deck of cards. This is to make it more difficult for cheats to synchronize with the random number generation applied to a single shuffle algorithm.

#### Run the simulation and examine the existing code in the application.

Start Visual Studio and open the solution `C:\Course\510D\Exercises\Ex121\Blackjack.sln`.

If necessary, select Test Explorer on the left side and then make sure everything is working correctly by running all tests. You should get all green lights.

Run the game and play a few hands just to see how it works.

<font color="red">**Don't get sidetracked here. Limit yourself to just a few minutes...dollars...**</font>

Open `CardLib.CardDeck.cs` for editing and move to the `Shuffle()` method. Briefly review what it does.

This is a classic "Simple Knuth Shuffler" that loops through the deck swapping two cards at a time.

Exit the game.

#### Implement the Strategy pattern.

In this section, you will implement the Strategy pattern by moving the shuffle algorithm into a separate class. This strategy class will implement an interface so that alternative strategies can be supported later.

Add a new class to the `CardLib` project by right-clicking the project, selecting **Add | New Item...**, and picking **Class**. Type the name ``KnuthShuffler`.

Change the class declaration to `public`.

Highlight and copy the entire `Shuffle()` method from `CardDeck`. Paste it into the `KnuthShuffler` class.

Change its return type to `void` and delete the statement `return this;`.

Modify the method to accept as a parameter a `List<PlayingCard>` called `deck`.

At this point your KnuthShuffler class should look like:

```C#
namespace CardLib
{
    public class KnuthShuffler
    {
        public void Shuffle(List<PlayingCard> deck)
        {
            for (int i = 0; i < deck.Count; i++)
            {
                int swapIndex = RandomNumber.Next(0, deck.Count);
                PlayingCard temp = deck[swapIndex];
                deck[swapIndex] = deck[i];
                deck[i] = temp;
            }
        }
    }
}
```

Run all tests to get green lights.

Single-click the `KnuthShuffler` class name and select **Extract Interface** from the light bulb drop down list on the left side. Name the interface `IShuffler` and make sure the `Shuffle(...)` method is checked. Click **OK**.

Note that the interface file has been added to the `CardLib` project and the `KnuthShuffler` specification indicates it implements this interface.

Return to editing `CardLib.CardDeck` and before the existing `Shuffle()` method, add an overload that accepts an `IShuffler` as a parameter. Call it `shuffler`.

Within this method call `Shuffler.Shuffle(deck);` and then `return this;`;

Your code should look something like:

```C#
public CardDeck Shuffle(IShuffler shuffler)
{
    shuffler.Shuffle(deck);     
    return this;
}
```

Get a clean compile.

#### Add a test to ensure the KnuthShuffler strategy is working.

Open `BlackjackTests.CardDeckTests.cs` and find the method `ShuffleTest()`.

Highlight and copy the entirety of this method and paste it back into the `CardDeckTests` class. Change its name to `ShuffleStrategyTest()`.

Within this test find the line that calls `deck.Shuffle()` and change it to be `deck.Shuffle(new KnuthShuffler());`.

Run all tests and ensure you get green lights that indicate the shuffle strategy is working.

#### Add a second shuffler to demonstrate that we can change the strategy as needed.

Open `KnuthShuffler.cs` for editing. Highlight and copy the entirety of the `KnuthShuffler` class. Paste it into the same file, and change its name to `ModifiedKnuthShuffler`.

Change the call to `RandomNumber.Next(0...` to be `RandomNumber.Next(i...`. This is the only difference between the two algorithms.

Get a clean compile.

Repeat the process you followed when adding `ShuffleStrategyTest` above, but this time name the method `ModifiedShuffleStrategyTest()` and pass in an instance of the `ModifiedKnuthShuffler` for the call to `Shuffle(...)`.

Get all green lights to ensure your second shuffle strategy is working.

At this point there are now two copies of the Knuth shuffle logic, one in the `KnuthShuffler` class and one still in the `CardDeck` class.

#### Refactor the `CardDeck` to call the `KnuthShuffler` for its default `Shuffle()` method.

<font color="red">**Do not violate the Open/Close principle while doing this.**</font>

Open `CardDeck` for editing and add a private field called `currentShuffler`. In its declaration instantiate and assign a `KnuthShuffler` to it.

Your field should be something like this (the lines before and after are shown).

```C#
private List<PlayingCard> deck = new List<PlayingCard>();

private IShuffler currentShuffler = new KnuthShuffler();

public int Count { 
    get 
    { 
        return deck.Count; 
    } 
}
```

Move to the default `Shuffle()` method and replace the logic with a call to `currentShuffler.Shuffle(deck)`;

Your `Shuffle()` method should now look like...

```C#
public CardDeck Shuffle()
{
    currentShuffler.Shuffle(deck); 
    return this;
}
```

This is a good time to run all tests and be sure you get all green lights.

## Congratulations! You have implemented the Strategy pattern in this application. Proceed to the Bonus if you have more time.

# Bonus (Optional)

Modify the `CardDeck` class and the user interface to allow the shuffler to be changed while playing the game.

Open `CardDeck` for editing and just before the `Shuffle(...)` methods, add a method called `ChangeShuffler(IShuffler shuffler)`. This method should assign `currentShuffler = shuffler`.

Your `ChangeShuffler(...)` method should look something like:

```C#
currentShuffler = shuffler;
```

Run all tests to ensure nothing has been broken.

In the `Blackjack` project, double-click the `OtherPreferencesDialog` form to open it in the design view.

Right-click the form and select **View Code**.

In the form load event, initialize the shuffler choices using``ddlSelectItem.Items.Add("Knuth Shuffler");` after the line that checks if the deck is null. Repeat for the Modified Knuth Shuffler.

Your method should look like this...

```C#
private void Form1_Load(object sender, EventArgs e)
{
   if (deck == null) 
       return; 
    
   ddlSelectItem.Items.Add("Knuth Shuffler"); 
   ddlSelectItem.Items.Add("Modified Knuth Shuffler");
}
```

In the `btnSelect_Click(...)` method, add a switch statement to check the `.SelectedIndex` property to see which shuffler was selected. Make a call to `deck.ChangeShuffler(...)` passing in a new instance of the selected shuffler as appropriate.

Your C# code might look like this:
  
```C#
private void btnSelect_Click(object sender, EventArgs e)
{
    switch (ddlSelectShuffler.SelectedIndex)
    {
        case 0: 
            { 
                deck.ChangeShuffler(new KnuthShuffler()); 
                break; 
            }
        
        case 1: 
            {
                deck.ChangeShuffler(new ModifiedKnuthShuffler()); 
                break; 
            }
    }
    
    this.Dispose();
}
```

Run all tests, then play the game. Choose each of the different shufflers and play a few hands.

#### Do some final refactoring to clean up the text in the `OtherPreferencesDialog` and the menu in the `MainGameTable.cs` form.

Do whatever you think is necessary. Keep running all tests and executing the game to ensure the refactoring did not break anything.

Before finishing:

- Run all tests and verify all lights are green
- Run the simulation and ensure it is still working (select different shufflers)

## Congratulations! You have finished the bonus.